In [9]:
import matplotlib.pyplot as plt
from scipy.io import wavfile

def graph_spectrogram(wav_file, img_file):
    rate, data = get_wav_info(wav_file)
    nfft = 256  # Length of the windowing segments
    fs = 256    # Sampling frequency
    pxx, freqs, bins, im = plt.specgram(data, nfft,fs)
    plt.axis('off')
    plt.savefig(img_file,
                dpi=100, # Dots per inch
                frameon='false',
                aspect='normal',
                bbox_inches='tight',
                pad_inches=0) # Spectrogram saved as a .png 

def get_wav_info(wav_file):
    rate, data = wavfile.read(wav_file)
    return rate, data

In [ ]:
import glob
import os

def generate_image(commend):
    src_dir = './warehouse/speech_commands/' + commend + '/'
    dst_dir = './warehouse/speech_commands_images/' + commend + '/'
    wav_list = glob.glob(src_dir + '*.wav')

    for it in wav_list:
        filename = os.path.basename(it)
        graph_spectrogram(src_dir + filename, dst_dir + filename + '.png')

/Users/tykimos/Projects/Keras/venv/lib/python2.7/site-packages/matplotlib/axes/_axes.py:7235: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)


In [ ]:
generate_image('go')

In [ ]:
generate_image('yes')
generate_image('no')
generate_image('up')
generate_image('down')
generate_image('left')
generate_image('right')
generate_image('on')
generate_image('off')
generate_image('go')
generate_image('zero')
generate_image('one')
generate_image('two')
generate_image('three')
generate_image('four')
generate_image('five')
generate_image('six')
generate_image('seven')
generate_image('eight')
generate_image('nine')

2372

2368

74

6 2 2

46 14 14

1472 448 448


In [ ]:
# 0. 사용할 패키지 불러오기
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

# 랜덤시드 고정시키기
np.random.seed(3)

# 1. 데이터 생성하기
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'warehouse/speech_commands_images/train',
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
        'warehouse/speech_commands_images/val',
        target_size=(256, 256),  
        batch_size=32,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'warehouse/speech_commands_images/test',
        target_size=(256, 256),  
        batch_size=32,
        class_mode='categorical')


# 2. 모델 구성하기
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# 3. 모델 학습과정 설정하기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=46,
        epochs=5,
        validation_data=val_generator,
        validation_steps=14)

# 6. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=14)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 7. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)